# Video Actor Synchroncy and Causality (VASC)
## RAEng: Measuring Responsive Caregiving Project
### Caspar Addyman, 2020
### https://github.com/infantlab/VASC

# Step 1  Process videos using OpenPose

This script uses [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) human figure recognition neural network to create labeled wireframes for each figure in each frame of a video. OpenPoseDemo will go through a video frame by frame outputing a JSON file for each frame that contains a set of coordinate points and for a wireframe for each video.

#### Before running
First we need to download and install the [OpenPoseDemo](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/demo_overview.md) executable.

Additionally, you need to download the trained neural-network models that OpenPose uses. To do this go to the `models` subdirectory of OpenPose directory, and double-click / run the `models.bat` script.

The `openposedemo` bin/exe file can be run manually from the command line. It is worth trying this first so you understand what  `openposedemo` is. See [this guide](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/demo_overview.md) or open a terminal app or Windows Powershell, navigate to the openpose installation folder and then try this command

```
:: Windows
bin\OpenPoseDemo.exe --video examples\media\video.avi --write_json output
# Mac/Linux
./build/examples/openpose/openpose.bin --video examples/media/video.avi --write_json output
```

#### Load python libraries

In [ ]:
#what python libraries to we need?
import os
import sys
import time
import glob
import json
from datetime import datetime

#### Where is OpenPose?

We need the full path to your openpose directory

In [ ]:
# location of openposedemo - THIS WILL BE DIFFERENT ON YOUR COMPUTER
openposepath = "C:\\Users\\cas\\openpose-1.5.0-binaries-win64-gpu-python-flir-3d_recommended\\"
#openposepath = "C:\\Users\\caspar\\openpose-1.4.0-win64-cpu-binaries\\"

if sys.platform == "win32":
    app = "bin\\OpenPoseDemo.exe"
else:
    app = 'bin\\OpenPoseDemo.bin'

openposeapp = openposepath + app
print(openposeapp)

#### Where are your videos?

In the next cell you need to specify the folder with your set of video files. So that we process them. These scripts use the following director structure. It expects your videos to be in a subfolder of your project 

```
path\to\project\myvideos
```

and then it creates a folder `out` in the project at the same level as the videos with three subfolders for JSON files, the aggregated timeseries and the analyses

```
path\to\project\out\openpose
path\to\project\out\timeseries
path\to\project\out\analyses
```

In [ ]:
# where's the project folder? (with trailing slash)
# projectpath = os.getcwd() + "\\..\\lookit\\"
projectpath = "C:\\Users\\Cas\\OneDrive - Goldsmiths College\\Projects\\Measuring Responsive Caregiving\\lookit\\"

# locations of videos and output
videos_in = projectpath 
videos_out_openpose   = projectpath + "out\\openpose"
videos_out_timeseries = projectpath + "out\\timeseries"
videos_out_analyses   = projectpath + "out\\analyses"

print(videos_in)
print(videos_out_openpose)
print(videos_out_timeseries)
print(videos_out_analyses)

In [ ]:
#first get list of videos in the inbox
avis = glob.glob(videos_in + "*.avi")
mp4s = glob.glob(videos_in + "*.mp4")

print("We found %d avis" % len(avis))
print("We found %d mp4s" % len(mp4s))

#For the moment we will manually specify what videos to process. 
#TODO generate a list of force or skip videos to automate things slightly
allvideos = []
allvideos.extend(avis)
allvideos.extend(mp4s)

#### Calling the OpenPose app
To operate OpenPose we pass a set of parameters to the demo executable. For the full list of options see  [OpenPoseDemo](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/demo_overview.md)

Our main parameters are

```
--video        path\to\video_to_process   #input video
--write_json   path\to\output_directory   #one json file per frame
--write_video  path\to\output_directory   #video with identified figures
--write_images path\to\output_directory   #one image per frame with wireframes
--disable_blending true/false             # wireframes on black background (true) or blended on top of video (false)
 ```

Other useful params
 ```
--frame_first  100    #start from frame 100
--display 0           #don't show the images as they are processed
 ```


In [ ]:
params = dict()
params["write_json"] = videos_out_openpose
params["write_images"] = videos_out_openpose  #for the moment dump images in output file - TODO name subfolder
params["disable_blending"] = "true"
params["display"]  = "1"

#### The main openpose loop

Call the openpose app for each of the videos at a time. For each one print the full command that we use so that you can use it manually to investigate any errors. 

Finally, we write a simple list of the processed videos to a file called `videos.json`.

In [ ]:
currdir =  os.getcwd() + "\\" #keep track of current directory so we can change back to it after processing

optstring = ""
for key in params:
    optstring += " --" + key +  ' "' + params[key] + '"' #need to quote paths 

print(optstring)

videosjson = {}
os.chdir(openposepath)
for vid in allvideos:
    #first we need base name of video for the output file name
    fullname = os.path.basename(vid)
    base, fmt = os.path.splitext(fullname)
    video_outname = base + "_output.avi"
    #log some info about this video
    videosjson[base] = {}
    videosjson[base]["fullname"] = fullname
    videosjson[base]["format"] = fmt
    videosjson[base]["openpose"] =  {"exitcode" : None, "when" : None} 

    print("\n\nStaring openpose processing of " + vid)
    try:
        # Log the time
        time_start = time.time()
        video = ' --video "' + vid + '"'
        video_out = ' --write_video "' + videos_out_openpose + '\\' + video_outname + '"'
        opbin = openposeapp + video + video_out + optstring
        print(opbin)
        exitcode = os.system(opbin)
        videosjson[base]["openpose"]["exitcode"] = exitcode
        # Log the time again
        time_end = time.time()
        if (exitcode == 0):
            videosjson[base]["openpose"]["when"] = datetime.now().isoformat()
            videosjson[base]["openpose"]["out"] = videos_out_openpose + '\\' + video_outname
            print ("Done " + vid)
            print ("It took %d seconds for conversion." % (time_end-time_start))
        else:
            print("OpenPose error. Exit code %d" % exitcode)
    except Exception as e:
        print("Error: ", e)
        pass
    
#change the directory back
os.chdir(currdir)
    
#now we've finished, write a list of processed videos to a file
with open(videos_out_openpose + '\\videos.json', 'w') as outfile:
    json.dump(videosjson, outfile)